In [1]:
!pip install transformers accelerate einops Xformers langchain InstructorEmbedding sentence-transformers chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install unstructured pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 526.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 

In [174]:
# CLEAR CUDA
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [175]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

generate_text = pipeline(model=model_name, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [176]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

In [46]:
hf_pipeline

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<instruct_pipeline.InstructionTextGenerationPipeline object at 0x7fc3d7c7be80>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None)

In [6]:
# TODO: Check if input is checked somewhere, otherwise check something like this:

# Your output sequence
output_sequence = "This is an example output sequence."

# Tokenize the output sequence
tokenized_output = tokenizer.encode(output_sequence, return_tensors="pt")

# Count the number of tokens in the tokenized sequence
num_tokens = tokenized_output.shape[1]

print(f"Number of tokens in the output sequence: {num_tokens}")

Number of tokens in the output sequence: 7


# Test LLM Chain

In [177]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

"\nDolly the sheep was the world's first successful cloned mammal. In 2008, scientists used DNA from a single cell to produce, for the first time, an entire sheep identical to a living sheep."

In [178]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

'\nThe primary colors are Red, Orange, Yellow, Green, Blue, Indigo, and Violet.'

# Prepare Documents


In [9]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 23.3 MB/s eta 0:00:00


In [10]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [179]:
# DOWNLOAD Workable template employee handbook from: https://resources.workable.com/wp-content/uploads/2017/09/Employee-Handbook.pdf

# langchain TUTORIAL WAY
# Load and process the PDF files
loader = DirectoryLoader('/content/docs/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [180]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [181]:
len(texts)

91

In [182]:
texts[2]

Document(page_content='This employee  handbook  defines  who we are and how we work  together.  we will do \neverything  possible  to create  a fair and productive  workplace,  but we need  your help.  \nWe’ve  created  this handbook  to guide  you. \n \nThis handbook  isn’t a contract  or a guarantee  of employment.  It’s a collection  of our \nexpectations,  commitments  and responsibilities.  Please  read this employee  handbook  \ncarefully  and consult  it whenever  you need  to.  \nGetting  to know  our company  \n[Provide  a brief history  of your company  here.  If possible,  use illustrations  and charts  to \nmake  this information  memorable.  Present  your company’s  culture  and vision.  Avoid  \njargon  and overused  phrases  (e.g. “competitive, ” “fast-paced  environment, ” “core \nvalues”  and “innovative. ”) Also,  it’d be a good  idea to include  quotes  and pictures  of \nyour company’ s founders,  CEO  or other  executives  to make  this introduction  more  \nperson

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [183]:
# Choose one of the top performers from the MTEB English Leaderboard

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

# Make a Vector Database

In [160]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# and a retriever

In [161]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})

In [162]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fc25ca82b00>, search_type='similarity', search_kwargs={'k': 2})

# Test the basic retriever

In [184]:
docs = retriever.get_relevant_documents("What is a full time employee?")

In [185]:
docs[0]

Document(page_content='your company’ s founders,  CEO  or other  executives  to make  this introduction  more  \npersonable. ] \nEmployment  basics  \nIn this section,  we explain  our employee  contract  types  and define  our basic  \nemployment  policies.   \nEmployment  contract  types  \nFull-time employees  work at least  [30 hours ] per week  or [130 hours ] per month  on \naverage.  \n \nPart-time employees  are those  who work  fewer  than [30 hours ] per week.  \n \nFull-time and part-time employees  can have  either  temporary  or indefinite  duration  \ncontracts.  Full-time employees  under  an indefinite  duration  contract  are entitled  to our \ncompany’s  full benefits  package.  \n \n[Insert  this if employees  are in the U.S: We remind  you that, in the U.S.,  employment  is \n“at-will.” This means  that you or our company  may terminate  our employment  \nrelationshi p at any time and for any non-discriminatory  reason(s). ]', metadata={'source': '/content/docs/Empl

# Make a proper Question Retrieval chain

In [186]:
# use map_reduce to prevent token mismatch errors?
# possible issue due to batch size?

qa_chain = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True)

In [187]:
qa_chain

RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="\nYou are a human resources specialist.\nUse the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nAlways answer with unbiased, ethical and safe advise.\n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers_modules.databricks.dolly-v2-3b.e19a5252f69d79d94ac95045eb9b8

In [188]:
qa_chain.combine_documents_chain.llm_chain.prompt

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="\nYou are a human resources specialist.\nUse the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nAlways answer with unbiased, ethical and safe advise.\n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True)

In [189]:
qa_chain.combine_documents_chain.llm_chain.prompt.template

"\nYou are a human resources specialist.\nUse the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nAlways answer with unbiased, ethical and safe advise.\n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [190]:
# try to set the tone to "as a human-like HR person"
qa_chain.combine_documents_chain.llm_chain.prompt.template = '''
You are a human resources specialist.
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always answer with unbiased, ethical and safe advise.
----------------
{context}

Question: {question}
Helpful Answer:'''

In [191]:
qa_chain.combine_documents_chain.llm_chain.prompt.template

"\nYou are a human resources specialist.\nUse the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nAlways answer with unbiased, ethical and safe advise.\n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [218]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
      print(source.metadata['source'] + ' Page ' + str(source.metadata['page']))

# Test QA Retrieving Chain

In [220]:
query = "What is a full-time employee?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


A full-time employee works at least 30 hours a week on average.


Sources:
/content/docs/Employee-Handbook.pdf Page 2
/content/docs/Employee-Handbook.pdf Page 22


In [221]:
query = "What holidays do the company observe?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


The holidays our company observes are:
- New Year’s Day
- Martin Luther King Day
- President’s Day
- Good Friday
- Memorial Day
- Independence Day
- Labor Day
- Columbus Day
- Veterans Day
- Thanksgiving Day
- Christmas Day


Sources:
/content/docs/Employee-Handbook.pdf Page 26
/content/docs/Employee-Handbook.pdf Page 27


In [222]:
query = "How long until I can get time off?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


It will depend on your contract with the company. You can check it online. It will
show you the right for you to get time off. In some countries, you may get greater
time off benefit than others.


Sources:
/content/docs/Employee-Handbook.pdf Page 27
/content/docs/Employee-Handbook.pdf Page 26


In [224]:
query = "I need to take a break"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Although we offer  long-term illness leave,  you should consult your  employment  contract 
for more information.  If you need  to take a break because you are suffering  from 
serious  illness,  you can also apply  for bereavement leave  from your human resources 
department.  They will process the application for you and will grant you  a minimum of 
three  days  of paid bereavement  leave.


Sources:
/content/docs/Employee-Handbook.pdf Page 27
/content/docs/Employee-Handbook.pdf Page 28


# Notes for next version
- Add memory
- Figure out how to handle data in databricks using dbfs ([Dolly Example](https://www.dbdemos.ai/demo-notebooks.html?demoName=llm-dolly-chatbot))

- Add docs such as:
  - onboarding
  - training program 
  - bylaws/regulations